In [3]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language

In [4]:
loader = GenericLoader.from_filesystem("/Users/mertbozkir/Code/langgraph-rag/langgraph/",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON),
)
documents = loader.load()
len(documents)

150

In [10]:
documents[4] # Example of the document

Document(page_content='class ChannelWrite(RunnablePassthrough):\n    channels: Sequence[tuple[str, Optional[Runnable]]]\n    """\n    Mapping of write channels to Runnables that return the value to be written,\n    or None to skip writing.\n    """\n\n    class Config:\n        arbitrary_types_allowed = True\n\n    def __init__(\n        self,\n        *,\n        channels: Sequence[tuple[str, Optional[Runnable]]],\n    ):\n        super().__init__(func=self._write, afunc=self._awrite, channels=channels)\n        self.name = f"ChannelWrite<{\',\'.join(chan for chan, _ in self.channels)}>"\n\n    def __repr_args__(self) -> Any:\n        return [("channels", self.channels)]\n\n    @property\n    def config_specs(self) -> list[ConfigurableFieldSpec]:\n        return [\n            ConfigurableFieldSpec(\n                id=CONFIG_KEY_SEND,\n                name=CONFIG_KEY_SEND,\n                description=None,\n                default=None,\n                annotation=None,\n           

In [21]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [22]:
# from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [25]:
query = "How can I create langgraph multi-agent workflow??"
docs = db.similarity_search(query)
print(docs[0].page_content)

def create_agent_executor(agent_runnable, tools, input_schema=None):
    if isinstance(tools, ToolExecutor):
        tool_executor = tools
    else:
        tool_executor = ToolExecutor(tools)

    state = _get_agent_state(input_schema)

    # Define logic that will be used to determine which conditional edge to go down

    def should_continue(data):
        # If the agent outcome is an AgentFinish, then we return `exit` string
        # This will be used when setting up the graph to define the flow
        if isinstance(data["agent_outcome"], AgentFinish):
            return "end"
        # Otherwise, an AgentAction is returned
        # Here we return `continue` string
        # This will be used when setting up the graph to define the flow
        else:
            return "continue"

    def run_agent(data):
        agent_outcome = agent_runnable.invoke(data)
        return {"agent_outcome": agent_outcome}

    async def arun_agent(data):
        agent_outcome = await agent_runnabl

In [26]:
from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import VectorStoreRetriever
retriever = VectorStoreRetriever(vectorstore=db)
retrievalQA = RetrievalQA.from_llm(llm=OpenAI(), retriever=retriever)

/Users/mertbozkir/Code/langgraph-rag/langrag/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [29]:
from langchain import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


template = """Just give me the clean instructions and the code!"""
   
memory = ConversationBufferMemory(
    memory_key='chat_history', return_messages=True,
)
prompt_template = PromptTemplate.from_template(template)

conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=OpenAI(),
        retriever=db.as_retriever(),
        memory=memory,
        condense_question_prompt=prompt_template,
)

In [30]:
question = "How can I create langgraph multi-agent workflow??"
result = conversation_chain(question)
result["answer"]

/Users/mertbozkir/Code/langgraph-rag/langrag/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


" To create a langgraph multi-agent workflow, you can use the `create_agent_executor` function. This function takes in an `agent_runnable` and a list of `tools`. It also has an optional `input_schema` parameter that can be used to define the initial state of the agent. The function will then create a `StateGraph` and define the logic for determining the next node to be called based on the agent's outcome. It will also handle the execution of `tools` and compile the workflow into a LangChain Runnable."

" To create a langgraph multi-agent workflow, you can use the `create_agent_executor` function. 
This function takes in an `agent_runnable` and a list of `tools`. 
It also has an optional `input_schema` parameter that can be used to define the initial state of the agent. 
The function will then create a `StateGraph` and define the logic for determining the next node to be called based on the agent's outcome. 
It will also handle the execution of `tools` and compile the workflow into a LangChain Runnable."